In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import random
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from scipy.stats import mode
import gc  # Import the garbage collection module
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, f1_score
import pickle
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv1D, Activation, Dropout, Dense, Flatten, MaxPooling1D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
import seaborn as sn
import pandas as pd
import sklearn.metrics as metrics
import keras

def sort_cmp (key):
    ans = 0
    for c in key:
        if (c.isdigit() == True):
            ans = ans * 10 + int (c)
    return ans

def load_test_data(dir_rec, save=False):
    x, y = [], [] 
    print (dir_rec)
    for file in sorted(glob.glob(dir_rec + "/*.wav")):
        # load an audio file as a floating point time series.    
        data, sr = librosa.load(file)
        #print ('shape:', data.shape, ' ', sr)
        # extract features from audio files into numpy array
        #feature = extract_feature(data, sr, mfcc=True/*, chroma=True, mel=True)
        seg = np.array (divide_into_segments (data, sr))
        i = 0
        for s in seg:
            feature = extract_feature(s, sr, mfcc=True, chroma=True, mel=True)
            x.append(feature)
            #x_orig.append (s)
            i+= 1
            if (i == 5):
                break
        file_name = os.path.basename(file)
        #print (file_name)
        if (file_name.count ('Neutral') > 0):
            emotion = file_name.split ('.')[0].split ('_')[4]
        else:
            emotion = file_name.split ('.')[0].split ('_')[4]
        
        if (emotion == 'Calmness'):
            emotion = 'Calmness'    
        # get emotion label from the file name
        i = 0
        for s in seg:
            y.append(emotions.index (emotion))
            i += 1
            if (i == 5):
                break
    
    if save==True:
        np.save('X', np.array(x))
        np.save('y', y)
        
    return np.array(x), y
def divide_into_segments (data, sr, seg_dur_seconds = 5):
    segment_length = sr * seg_dur_seconds
    num_sections = int(np.floor(len(data) / segment_length))
    split = []
    for i in range(num_sections):
        t = data[i * segment_length: (i + 1) * segment_length]
        split.append(t)
    return split 

def extract_feature(data, sr, mfcc, chroma, mel):
    if chroma:                          
        stft = np.abs(librosa.stft(data))  
    result = np.array([])
    if mfcc:                          
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:                          
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T,axis=0)
        result = np.hstack((result, chroma))
        
    if mel:           
        mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sr).T,axis=0)
        result = np.hstack((result, mel))
        
    return result 

def proc_data (file, emotion):
    x = []
    y = []
    data, sr = librosa.load(file)
    seg = np.array (divide_into_segments (data, sr))
    i = 0
    for s in seg:
        feature = extract_feature(s, sr, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        i+= 1
        if (i == 5):
            break
    i = 0
    for s in seg:
        y.append(emotions.index (emotion))
        i += 1
        if (i == 5):
            break
    return x, y

def extract_index_video(filename):
    key = (int (filename.split ('/')[-1].split('_')[1]), int (filename.split ('/')[-1].split('_')[5]))
    return key

def extract_index_audio(filename):
    if (filename.count ('neutral') > 0):
        return (int (filename.split ('/')[-1].split('_')[1]), 0)
    
    key = (int (filename.split ('/')[-1].split('_')[1]), int (filename.split ('/')[-1].split('_')[3]))
    
    return key

acc = []
fscore = []
confusion = []

root_folder = "../Dropbox/EAV"

emotions = ["Anger", "Neutral", "Sadness", "Calmness", "Happiness"]
for subfolder in sorted(os.listdir(root_folder), key = sort_cmp):
    print(subfolder)
    categorized_files = {emotion: [] for emotion in emotions}
    subfolder_path = os.path.join(root_folder, subfolder)
    audio_path = os.path.join(subfolder_path, 'Audio')
    data, labels = [], []
    if os.path.exists(audio_path):
        all_files = sorted(os.listdir(audio_path))
        for file in all_files:
            if file.endswith(".wav"):
                for emotion in emotions:
                    if emotion in file:
                        if (emotion != 'Neutral' and file.count ('Neutral') > 0):
                            continue
                        audio_path2 = os.path.join(audio_path, file)
                        categorized_files[emotion].append(audio_path2)
        
        X, y = load_test_data(audio_path)
        idx = 0
        
        for class_index, emotion in enumerate(emotions):
            for file_index, audio_path2 in enumerate(categorized_files[emotion]):
                x,y = proc_data (audio_path2, emotion)        
                data.extend(x)
                labels.extend(y)
                idx+=1

        x_train = []
        y_train = []
        x_test = []
        y_test = []
        
        for i, emotion in enumerate (emotions):
            x_train.extend (data [i*80: i*80 + 40])
            y_train.extend (labels [i*80: i*80 + 40])
            x_test.extend (data [i*80 + 40: i*80 + 80])
            y_test.extend (labels [i*80 + 40: i*80 + 80])
        
        x_train = np.array(x_train)
        x_test = np.array(x_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        x_train = np.expand_dims(x_train, axis=2)
        x_test = np.expand_dims(x_test, axis=2)
         
        identity_matrix = np.eye(len(emotions))
        train_labels_one_hot = np.array([identity_matrix[label] for label in y_train])
        test_labels_one_hot = np.array([identity_matrix[label] for label in y_test])
        
        audio_model = Sequential()
        audio_model.add(Conv1D(256, 5,padding='same', input_shape=(180,1))) # 1st layer
        audio_model.add(Activation('relu'))
        audio_model.add(Conv1D(128, 5,padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))) # 2nd layer
        audio_model.add(Activation('relu'))
        audio_model.add(Dropout(0.1))
        audio_model.add(MaxPooling1D(pool_size=(8)))
        audio_model.add(Conv1D(128, 5,padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))) # 3rd layer
        audio_model.add(Activation('relu'))
        audio_model.add(Conv1D(128, 5,padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))) # 4th layer
        audio_model.add(Activation('relu'))
        audio_model.add(Dropout(0.5))
        audio_model.add(Flatten())
        audio_model.add(Dense(units=5,
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-5)
                        )
        ) # 7th layer
        audio_model.add(Activation('softmax'))
        audio_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        audio_model.fit(x_train, train_labels_one_hot, validation_data=(x_test, test_labels_one_hot), epochs=100, batch_size=64, verbose = False)
        loss, accuracy = audio_model.evaluate(x_test, test_labels_one_hot)
        y_pred = audio_model.predict (x_test)
        f1 = f1_score(y_test,np.argmax(y_pred, axis=-1),average='weighted')
        acc.append (accuracy)
        fscore.append (f1)
        confusion_emotions = ["Anger", "Neutral", "Sadness", "Calmness", "Happiness"]
        cm = metrics.confusion_matrix(y_test, np.argmax(y_pred, axis=-1))
        confusion.append (cm)
        print(f"Audio Accuracy for {subfolder} : {accuracy * 100:.2f}%")
        print ('F1: ', f1)
        print ('Confusion:', cm)